#Understanding per-example feature importance

In [3]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set_context('notebook')
sns.set_style('darkgrid')
import sklearn as skl
import sklearn.metrics
import sklearn.learning_curve
import sklearn.ensemble
import sklearn.dummy
import sklearn.grid_search
import sklearn.feature_extraction
import sklearn.feature_extraction.text
import numpy as np
import pickle

import sys
sys.path = sys.path + ['..']
import get_airbnb_data
import db
import learning
import imp
imp.reload(learning)
from learning import *

In [4]:
clf = pickle.load(open('../pipe2.pkl', 'rb'))

In [5]:
engine = db.create_root_engine()
rawtable = pd.io.sql.read_sql_table('listings', engine, index_col='id')
Xtr, Xte, ytr, yte = get_training_test_set(rawtable, make_features=make_features5, 
                                               categorize_rating=categorize_rating5)

In [25]:
single_example = Xtr.iloc[0, :]
comparison_examples = Xtr.iloc[1:101, :].reset_index(drop=True)  # Will need to select these randomly
comparison_examples

,amenity0,amenity1,amenity2,amenity3,amenity4,amenity5,amenity6,amenity7,amenity8,amenity9,...,features2_categorical13,features2_categorical14,features2_categorical15,features2_categorical16,features2_categorical17,features2_categorical18,nDescriptionMisspellings,homeValueIndex,medianListPricePerSqFt,medianSingleFamilyHomeValue
0,1,0,0,0,1,0,0,0,1,0,...,0,0,0,1,0,0,17,450600,534,0
1,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,7,0,276,0
2,1,0,0,0,1,0,0,0,1,0,...,0,1,0,0,1,0,1,150600,120,178300
3,1,0,1,0,1,0,1,0,1,1,...,0,0,0,1,0,0,12,433900,360,532800
4,1,0,0,0,1,0,0,0,0,0,...,0,1,0,1,0,0,8,177000,148,182300
5,1,0,0,0,1,0,0,0,1,0,...,0,1,0,1,0,0,10,0,234,0
6,1,1,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,10,1592200,1104,1910200
7,1,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,0,16,1040900,1024,3795100
8,1,1,0,0,1,0,0,0,0,0,...,0,1,0,0,1,0,7,223700,197,228200
9,1,0,1,0,1,0,0,0,1,0,...,0,0,0,0,1,0,10,0,0,0


In [9]:
single_proba = clf.predict_proba(single_example)
class_ = np.argmax(single_proba)

In [18]:
np.repeat(0, 100)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [29]:
Xtr.columns.get_loc('lat')

62

In [44]:
Xtr.columns[52]

'beds'

In [42]:
column = 52

In [45]:
single_example

amenity0                          1
amenity1                          0
amenity2                          1
amenity3                          0
amenity4                          0
amenity5                          0
amenity6                          0
amenity7                          0
amenity8                          0
amenity9                          0
amenity10                         0
amenity11                         0
amenity12                         0
amenity13                         0
amenity14                         0
amenity15                         1
amenity16                         0
amenity17                         0
amenity18                         0
amenity19                         0
amenity20                         0
amenity21                         1
amenity22                         1
amenity23                         0
amenity24                         0
amenity25                         0
amenity26                         0
amenity27                   

In [ ]:
importances = []
for column in range(97):
    # Need to reset index because we are combining data with different id so don't want to align by index
    scrambled_examples = pd.DataFrame(single_example, copy=True).T.iloc[np.repeat(0, 100), :].reset_index(drop=True)
    scrambled_examples.iloc[:, column] = comparison_examples.iloc[:, column]
# Either big spread or mean offset of scrambled predictions - normal prediction means feature was important
#   so take RMS
    importances.append(np.mean((clf.predict_proba(scrambled_examples)[:, class_] - single_proba[0, class_]) ** 2))

In [48]:
for i, importance in enumerate(importances):
    print(Xtr.columns[i], importance)

amenity0 0.0
amenity1 5.44730374713e-06
amenity2 0.000527626078011
amenity3 0.0
amenity4 0.0012527669955
amenity5 0.0
amenity6 6.60020407763e-05
amenity7 0.0
amenity8 8.62894843487e-05
amenity9 3.54080192358e-05
amenity10 6.01728152064e-05
amenity11 3.59734442324e-05
amenity12 0.0
amenity13 6.85228672742e-05
amenity14 0.0
amenity15 5.9956659417e-06
amenity16 0.000487548816954
amenity17 0.000362330174611
amenity18 1.75976705054e-05
amenity19 0.000162455500194
amenity20 1.56889971543e-05
amenity21 0.000566802126192
amenity22 0.0
amenity23 0.0
amenity24 0.0
amenity25 0.0
amenity26 0.0
amenity27 0.0
amenity28 0.0
amenity29 0.0
amenity30 0.0
amenity31 0.0
amenity32 0.0
amenity33 0.0
amenity34 0.0
amenity35 0.0
amenity36 0.0
amenity37 0.0
amenity38 0.0
amenity39 0.0
amenity40 0.0
amenity41 0.0
amenity42 0.0
amenity43 0.0
amenity44 0.0
amenity45 0.0
amenity46 0.0
amenity47 0.0
amenity48 0.0
amenity49 0.0
bathrooms 4.16533695046e-05
bedrooms 6.00108301707e-05
beds 0.00128386487085
instantBooka